In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [8]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [60]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,clusters):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth == 2:
            cluster_ids.append(key)
            cluster_structure[key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['f1','g','recall','precision','pf']
    for goal in goals:
        count = 0
        count_not = 0
        count_yes = 0
        precentage = 0
        score = []
        score_med = []
        cluster_info = {}
        for cluster in clusters:
            df = pd.read_csv(cluster + '/1385_LR_bellwether_' + goal + '.csv')
            counts = []
            med_count = []
            for row in range(df.shape[0]):
                j = df.iloc[row].values[1:]
                j_med = np.median(j)
                if goal != 'pf':
                    value = sum(i >= 0.66 for i in j)
                else:
                    value = sum(i <= 0.33 for i in j)
                med_count.append(j_med)
                counts.append(value)
            df['>=0.6'] = counts
            precentage = max(counts)/df.shape[0]
            x = int(cluster.rsplit('/',1)[1])
            #score.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),max(counts),round(np.mean(counts),2),df.iloc[counts.index(max(counts)),0]])
            if goal == 'pf':
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.62),counts[med_count.index(min(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(min(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])     
            elif goal == 'g':
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.16),counts[med_count.index(max(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(max(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])
            else:
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),counts[med_count.index(max(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(max(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])
            count += max(counts)
        score_df = pd.DataFrame(score_med, columns = ['id','expected','max_bell','avge','median','bellwether','size'])
        score_df = score_df.sort_values('id')
        score_df.to_csv(data_source1 + '/bellwether_' + goal + '.csv')

In [51]:
self_df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/projects/self_test.csv')
self_df = self_df[self_df['project'].isin(list(df_train.index))]
self_df.to_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/projects/self_test_modeified.csv')



In [61]:
for i in range(10):
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,clusters)